# This document demonstrates feature extractions process

We can extract object features **(e.g. area, length, width, circularity, pixel intensity, etc)** over multiple frames through which the object is tracked, and generate some plots to visualize whether or not these features are consistent across frames, and what the distribution is for these features across objects and biological datasets.

For example:
 
    Line plots where the x-axis is the index for the frame for each object (1, 2, 3, ...) and the y-axis is the feature. We can then see fluctuations in the feature measurement, which may indicate additional tracking and segmentation errors to investigate. 

    Histograms of the median feature value across frames for each object, so we can see whether objects fall into a unimodal or bimodal distribution, and whether there are outliers

    Boxplots comparing the feature values between different biological datasets (e.g. hTERT, SiHa, mouse liver)

## functions 

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from skimage import measure
from scipy.spatial import distance
from scipy.ndimage import center_of_mass
import tifffile as tiff
import numpy as np
from skimage.measure import regionprops
from skimage import img_as_ubyte
from PIL import Image
import matplotlib.pyplot as plt
import diplib as dip
import sys
from skimage.measure import label as sk_label
import numpy as np
from skimage.feature import graycomatrix, graycoprops

def get_diameter_cellenONE(frame, track_id, um_per_pixel = 0.69):
    area = np.sum(frame == track_id)
    return 2 * np.sqrt(area / np.pi) * um_per_pixel

def get_elongation_cellenONE(frame, track_id):
    # Create a binary mask for the given track ID
    binary_mask = (frame == track_id).astype(np.uint8)
    
    # Label the binary mask
    labeled_mask = sk_label(binary_mask)
    
    # Extract region properties
    regions = regionprops(labeled_mask)
    
    if len(regions) == 0:
        raise ValueError(f"No object with track ID {track_id} found in the frame.")
    
    region = regions[0]
    
    if region.minor_axis_length == 0: 
        print("The region minor axis lnegth is zero, we retrun zero to avoid zero division error")
        return 0

    
    return region.major_axis_length / region.minor_axis_length

def get_gray_intensity_cellenONE(frame, intensity_image, track_id):
    """
    Calculate the mean normalized intensity of the object with the given track ID in the frame.

    Parameters:
    - frame: numpy array, the pixel matrix where entries are object assignments.
    - intensity_image: numpy array, the intensity values of the corresponding pixels.
    - track_id: int, the ID of the track for which to calculate the intensity.

    Returns:
    - mean_intensity: float, the mean normalized intensity of the object.
    """
    # Create a binary mask for the given track ID
    binary_mask = (frame == track_id).astype(np.uint8)
    
    # Label the binary mask
    labeled_mask = sk_label(binary_mask)
    
    # Extract region properties
    regions = regionprops(labeled_mask, intensity_image=intensity_image)
    
    if len(regions) == 0:
        raise ValueError(f"No object with track ID {track_id} found in the frame.")
    
    region = regions[0]
    
    # Get the intensity values of the pixels inside the object
    intensity_values = region.intensity_image[region.image]
    
    return np.mean(intensity_values)


def get_glcm_features(frame, intensity_image, track_id, get_e):
    """
    Calculate the GLCM features of the object with the given track ID in the frame.

    Parameters:
    - frame: numpy array, the pixel matrix where entries are object assignments.
    - intensity_image: numpy array, the intensity values of the corresponding pixels.
    - track_id: int, the ID of the track for which to calculate the GLCM features.

    Returns:
    - glcm_features: tuple, containing GLCM features like energy and homogeneity.
    """
    # Create a binary mask for the given track ID
    binary_mask = (frame == track_id).astype(np.uint8)
    
    # Label the binary mask
    labeled_mask = sk_label(binary_mask)
    
    # Extract region properties
    regions = regionprops(labeled_mask, intensity_image=intensity_image)
    
    if len(regions) == 0:
        raise ValueError(f"No object with track ID {track_id} found in the frame.")
    
    region = regions[0]
    
    # Get the intensity values of the pixels inside the object
    intensity_values = region.intensity_image[region.image]
    
    # Normalize intensity values to the range [0, 255]
    intensity_values = np.round((intensity_values - np.min(intensity_values)) / (np.max(intensity_values) - np.min(intensity_values)) * 255).astype(np.uint8)
    
    # Reshape intensity values to 2D if they are not already
    if intensity_values.ndim == 1:
        size = int(np.sqrt(intensity_values.size))
        intensity_values = intensity_values[:size*size].reshape((size, size))
    
    # Calculate GLCM
    glcm = graycomatrix(intensity_values, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], 256, symmetric=True, normed=True)
    
    # Calculate and return GLCM features
    energy = graycoprops(glcm, 'energy').mean()
    homogeneity = graycoprops(glcm, 'homogeneity').mean()
    
    if get_e: return energy
    return homogeneity


def get_w(frame, track_id):
    """
    Converts a binary mask to a bounding box.

    :param numpy.ndarray mask: Binary mask.
    :return: Bounding box in the format (x, y, w, h).
    :rtype: list[int]
    """
    rows, cols = np.where(frame == track_id)
    x1, x2 = np.min(cols), np.max(cols)
    y1, y2 = np.min(rows), np.max(rows)
    return x2-x1

def get_h(frame, track_id):
    """
    Converts a binary mask to a bounding box.

    :param numpy.ndarray mask: Binary mask.
    :return: Bounding box in the format (x, y, w, h).
    :rtype: list[int]
    """
    #print(f'These are the unique values of thje frame: {np.unique(frame)}')

    rows, cols = np.where(frame == track_id)
    x1, x2 = np.min(cols), np.max(cols)
    y1, y2 = np.min(rows), np.max(rows)
    return y2-y1

def get_circ(frame, track_id):
    """
    Calculate the circularity and eccentricity of the object with the given track ID in the frame.

    Parameters:
    - frame: numpy array, the pixel matrix where entries are object assignments.
    - track_id: int, the ID of the track for which to calculate the features.

    Returns:
    - circularity: float, the circularity of the object. It is computed by the ratio StD/Mean of the Radius feature, and is 0 for a perfect circle
    - roundness: float, the eccentricity of the object. This measure is in the range (0,1], with 1 for a perfect circle.
    """
    '''# Create a binary mask for the given track ID
    binary_mask = (frame == track_id).astype(np.uint8)
    
    # Label the binary mask
    labeled_mask = label(binary_mask)
    
    # Extract region properties
    regions = regionprops(labeled_mask)
    
    if len(regions) == 0:
        raise ValueError(f"No object with track ID {track_id} found in the frame.")
    
    region = regions[0]
    
    # Calculate circularity
    area = region.area
    perimeter = region.perimeter
    circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter != 0 else 0
    
    # Calculate eccentricity
    eccentricity = region.eccentricity'''

    # Create a binary mask for the given track ID
    binary_mask = (frame == track_id).astype(np.uint8)
    
    # Use DIPlib for advanced measurement
    labels = dip.Label(binary_mask > 0)
    msr = dip.MeasurementTool.Measure(labels, features=["Perimeter", "Size", "Roundness", "Circularity"])

    # Extract measurements
    roundness = msr[1]["Roundness"][0]
    circularity = msr[1]["Circularity"][0]
    
    return roundness

def get_intensity(frame, intensity_image, track_id):
    """
    Calculate the mean normalized intensity of the object with the given track ID in the frame.

    Parameters:
    - frame: numpy array, the pixel matrix where entries are object assignments.
    - intensity_image: numpy array, the intensity values of the corresponding pixels.
    - track_id: int, the ID of the track for which to calculate the intensity.

    Returns:
    - mean_intensity: float, the mean normalized intensity of the object.
    """
    # Create a binary mask for the given track ID
    binary_mask = (frame == track_id).astype(np.uint8)
    
    # Label the binary mask
    labeled_mask = sk_label(binary_mask)
    
    # Extract region properties
    regions = regionprops(labeled_mask, intensity_image=intensity_image)
    
    if len(regions) == 0:
        raise ValueError(f"No object with track ID {track_id} found in the frame.")
    
    region = regions[0]
    
    # Get the intensity values of the pixels inside the object
    intensity_values = region.intensity_image[region.image]
    
    # Calculate the mean and standard deviation for the entire intensity image
    image_mean = np.mean(intensity_image)
    image_std = np.std(intensity_image)
    
    # Normalize the intensity values using Z-score normalization
    normalized_intensity_values = (intensity_values - image_mean) / image_std
    
    # Calculate the mean normalized intensity
    mean_intensity = np.mean(normalized_intensity_values)
    
    return mean_intensity


def linear_plot_with_summary(values, names, title, output_directory=None, xlab = 'Track ID'):
    """
    Plot a linear graph of values with summary statistics in the legend and dotted lines for bounds.

    Parameters:
    - values: list or numpy array of numerical values.
    - names: list of str, names corresponding to each value.
    - title: str, title of the plot.
    - output_directory: str, directory to save the plot.
    """
    values = np.array(values)
    names = np.array(names)
    
    # Calculate summary statistics
    mean_val = np.mean(values)
    median_val = np.median(values)
    min_val = np.min(values)
    max_val = np.max(values)
    std_val = np.std(values)
    q1_val = np.percentile(values, 25)
    q3_val = np.percentile(values, 75)
    iqr_val = q3_val - q1_val
    lower_bound = max(0, q1_val - 1.5 * iqr_val)
    upper_bound = q3_val + 1.5 * iqr_val
    
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot the values
    ax.plot(names, values, marker='o', linestyle='-', color='b', label=title)
    
    # Plot bounds as solid lines
    ax.axhline(lower_bound, color='cyan', linestyle='-', linewidth=2, label=f'Lower Bound: {lower_bound:.2f}')
    ax.axhline(upper_bound, color='magenta', linestyle='-', linewidth=2, label=f'Upper Bound: {upper_bound:.2f}')
    
    # Add summary statistics to the legend
    summary_stats = (f'Mean: {mean_val:.2f}\n'
                     f'Median: {median_val:.2f}\n'
                     f'Std: {std_val:.2f}\n'
                     f'Min: {min_val:.2f}\n'
                     f'Max: {max_val:.2f}')
    ax.plot([], [], ' ', label=summary_stats)
    
    # Highlight points outside the bounds and add their names
    for i, (value, name) in enumerate(zip(values, names)):
        if value < lower_bound or value > upper_bound:
            ax.annotate(name, (name, value), textcoords="offset points", xytext=(0,10), ha='center', fontsize=9, color='red')
    
    # Set the title and labels
    ax.set_title(f"Distribution of Track {title}", fontsize=16, fontweight='bold')
    ax.set_xlabel(xlab, fontsize=14)
    ax.set_ylabel(title, fontsize=14)
    
    # Customize ticks
    ax.tick_params(axis='both', which='major', labelsize=12)
    
    # Add a grid
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Add a legend
    ax.legend(fontsize=12, loc='upper left', bbox_to_anchor=(1, 1))
    
    # Show the plot
    plt.tight_layout()
    if output_directory is not None:
        os.makedirs(output_directory, exist_ok=True)
        plt.savefig(os.path.join(output_directory, f'{title}_linear_plot.png'), dpi=300)
    plt.show()


import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

def histogram_with_best_fit(values, title, xlab, output_directory=None, dataset = None):
    """
    Plot a histogram with a best-fit line and summary statistics in the legend.
    
    Parameters:
    - values: list or numpy array of numerical values.
    - title: str, title of the plot.
    - output_directory: str, directory to save the plot.
    """
    values = np.array(values)
    
    # Calculate summary statistics
    mean_val = np.mean(values)
    median_val = np.median(values)
    min_val = np.min(values)
    max_val = np.max(values)
    std_val = np.std(values)
    
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot the histogram
    count, bins, ignored = plt.hist(values, bins=30, density=False, alpha=0.6, color='b', label=title)
    
    # Plot the best-fit line
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mean_val, std_val)
    plt.plot(x, p * len(values) * (bins[1] - bins[0]), 'k', linewidth=2, label='Best Fit Line')
    
    # Add summary statistics to the legend
    summary_stats = (f'Mean: {mean_val:.2f}\n'
                     f'Median: {median_val:.2f}\n'
                     f'Std: {std_val:.2f}\n'
                     f'Min: {min_val:.2f}\n'
                     f'Max: {max_val:.2f}')
    ax.plot([], [], ' ', label=summary_stats)
    
    # Set the title and labels
    ax.set_title(f"Distribution of {dataset} {title}", fontsize=16, fontweight='bold')
    ax.set_xlabel(xlab, fontsize=14)
    ax.set_ylabel('Frequency', fontsize=14)
    
    # Customize ticks
    ax.tick_params(axis='both', which='major', labelsize=12)
    
    # Add a grid
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Add a legend
    ax.legend(fontsize=12, loc='upper left', bbox_to_anchor=(1, 1))
    
    # Show the plot
    plt.tight_layout()
    if output_directory is not None:
        os.makedirs(output_directory, exist_ok=True)
        plt.savefig(os.path.join(output_directory, f'{dataset}_{title}_histogram.png'), dpi=300)
    plt.show()

import matplotlib.pyplot as plt

def create_boxplot(data, xticklabels, title, output_directory=None):

    if len(data) != len(xticklabels):
        raise ValueError (f"Within the boxplot creating function, the length of the input value and the x labels are not the identical: {len(data)}, {len(xticklabels)}")
    
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Create the boxplot
    ax.boxplot(data)
    
    # Set the x-axis labels and ticks
    ax.set_xticklabels(xticklabels)
    ax.set_xlabel("Datasets")
    ax.set_title(f"Boxplot of {title}", fontsize=16, fontweight='bold')
    
    # Add grid
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Show the plot
    plt.tight_layout()
    if output_directory is not None:
        os.makedirs(output_directory, exist_ok=True)
        plt.savefig(os.path.join(output_directory, f'{title}_boxplot.png'), dpi=300)
    plt.show()


def create_violinplot(data, xticklabels, title, output_directory=None):
    """
    Create a violin plot for multiple lists of values.

    Parameters:
    - data: List of lists containing numerical values to plot.
    - xticklabels: Labels for the x-axis corresponding to each list in data.
    - title: Title of the plot.
    - output_directory: Directory to save the plot (default is None).
    """
    if len(data) != len(xticklabels):
        raise ValueError(f"Within the violin plot creating function, the length of the input value and the x labels are not identical: {len(data)}, {len(xticklabels)}")
    
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Create the violin plot
    parts = ax.violinplot(data, showmeans=True, showmedians=True)
    
    # Set the x-axis labels and ticks
    ax.set_xticks(range(1, len(xticklabels) + 1))
    ax.set_xticklabels(xticklabels)
    ax.set_xlabel("Datasets")
    ax.set_title(f"Violin Plot of {title}", fontsize=16, fontweight='bold')
    
    # Customize the plot
    for partname in ('cbars', 'cmins', 'cmaxes', 'cmeans', 'cmedians'):
        vp = parts[partname]
        vp.set_edgecolor('black')
        vp.set_linewidth(1)
    
    # Add grid
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Show the plot
    plt.tight_layout()
    if output_directory is not None:
        os.makedirs(output_directory, exist_ok=True)
        plt.savefig(os.path.join(output_directory, f'{title}_violinplot.png'), dpi=300)
    plt.show()

def create_violinplot_with_boxplot(data, xticklabels, title, output_directory=None):
    """
    Create a violin plot with embedded boxplots for multiple lists of values.

    Parameters:
    - data: List of lists containing numerical values to plot.
    - xticklabels: Labels for the x-axis corresponding to each list in data.
    - title: Title of the plot.
    - output_directory: Directory to save the plot (default is None).
    """
    if len(data) != len(xticklabels):
        raise ValueError(f"Within the violin plot creating function, the length of the input value and the x labels are not identical: {len(data)}, {len(xticklabels)}")
    
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Create the violin plot
    parts = ax.violinplot(data, showmeans=False, showmedians=False, showextrema=False)
    
    # Set the x-axis labels and ticks
    #ax.set_xticks(range(1, len(xticklabels) + 1))
    ax.set_xticklabels(xticklabels)
    ax.set_xlabel("Datasets")
    ax.set_title(f"Violin Plot of {title} across datasets", fontsize=16, fontweight='bold')
    
    # Customize the violin plot
    for partname in ('cbars', 'cmins', 'cmaxes', 'cmeans', 'cmedians'):
        if partname in parts:
            vp = parts[partname]
            vp.set_edgecolor('black')
            vp.set_linewidth(1)
    
    # Add the boxplot inside the violin plot
    boxprops = dict(linestyle='-', linewidth=1.5, color='black')
    whiskerprops = dict(linestyle='-', linewidth=1.5, color='black')
    capprops = dict(linestyle='-', linewidth=1.5, color='black')
    medianprops = dict(linestyle='-', linewidth=1.5, color='red')
    meanprops = dict(linestyle='--', linewidth=1.5, color='blue')
    
    ax.boxplot(data, positions=np.arange(1, len(data) + 1), widths=0.1,
               boxprops=boxprops, whiskerprops=whiskerprops,
               capprops=capprops, medianprops=medianprops,
               meanprops=meanprops, showmeans=True, patch_artist=True, showfliers=True, flierprops=dict(marker='o', color='red', markersize=4))
    
    # Add grid
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Show the plot
    plt.tight_layout()
    if output_directory is not None:
        os.makedirs(output_directory, exist_ok=True)
        plt.savefig(os.path.join(output_directory, f'{title}_violinplot.png'), dpi=300)
    plt.show()

def feature_extraction_across_tracks(track_info_file, tif_directory, png_directory, features = ['area', 'length', 'width', 'circularity', 'intensity', 'energy', 'homogeneity', 'diameter', 'elongation', 'intensity_not_norm'], track = 'all', track_ids = None, output_directory = None, display_plot = True, dataset = None):

    # Load track information
    track_info = pd.read_csv(track_info_file, sep='\s+', header=None, names=['Track_ID', 'Start', 'End', 'Parent'], dtype={'Track_ID': int, 'Start': int, 'End': int, 'Parent': int})

    if track == 'all':
        track_ids = list(sorted(track_info['Track_ID']))
        if track_ids is None: raise RuntimeError("The track ID is None:(")

    area = [] 
    length = []
    width  = []
    circularity = []
    intensity = []
    energy = []
    homogeneity = []
    diameter = []
    elongation = []
    intensity_not_norm = []
    track_ids_to_save = []

    for i in track_ids:
        if i not in track_info['Track_ID'].values: 
            print(f"Track id {i} does not exists!!!!!!!!!!!!!!! ")
            continue
        track_id, start_frame, end_frame, parent_id = track_info.loc[track_info['Track_ID'].values == i].values[0]
        #print(f"the current track ID is: {track_id}")

        # Load the frames for the current track
        track_frames = []
        intensity_frames = []
        for frame in range(start_frame, end_frame+1):
            temp = tiff.imread(os.path.join(tif_directory, f'man_track{frame:04d}.tif'))
            if track_id not in np.unique(temp):
                #print(f"The track id: {track_id} is not in the frame {frame}")
                continue
            track_frames.append(tiff.imread(os.path.join(tif_directory, f'man_track{frame:04d}.tif')))
            intensity_frame = np.array(Image.open(os.path.join(png_directory, f'man_track{frame:04d}.png')).convert('L'))
            w =int((intensity_frame.shape[1]-10)/2)
            intensity_frames.append(intensity_frame[:,:w])

        #track_frames = [tiff.imread(os.path.join(tif_directory, f'man_track{frame:04d}.tif')) for frame in range(start_frame, end_frame + 1)]
        #intensity_frames = [np.array(Image.open(os.path.join(png_directory, f'man_track{frame:04d}.png')).convert('L'))[:,:192] for frame in range(start_frame, end_frame + 1)]
        #print(f"The shape of track is {track_frames[0].shape} and the intensity is {intensity_frames[0].shape}")
        #print(f"There are {len(track_frames)} which are")
        # Calculate the number of frames
        num_frames = len(track_frames)

        if 'area' in features:
            if np.isnan(np.median([np.sum(frame == track_id) for frame in track_frames])): 
                #print(f"The size of the frames is {len(track_frames)} with start and end frames {start_frame, end_frame}")
                #print(f"The track id is {track_id}")
                return None
            area.append(np.median([np.sum(frame == track_id) for frame in track_frames]))
        if 'length' in features:
            length.append(np.median([get_h(frame, track_id) for frame in track_frames]))
        if 'width' in features:
            width.append(np.median([get_w(frame, track_id) for frame in track_frames]))
        if 'circularity' in features: 
            circularity.append(np.median([get_circ(frame, track_id) for frame in track_frames]))
        if 'intensity' in features: 
            intensity.append(np.median([get_intensity(frame = track_frame, intensity_image = intensity_frame, track_id = track_id) for track_frame, intensity_frame in zip(track_frames, intensity_frames)]))
        if 'energy' in features: 
            energy.append(np.median([get_glcm_features(frame = track_frame, intensity_image = intensity_frame, track_id = track_id, get_e = True) for track_frame, intensity_frame in zip(track_frames, intensity_frames)]))
        if 'homogeneity' in features: 
            homogeneity.append(np.median([get_glcm_features(frame = track_frame, intensity_image = intensity_frame, track_id = track_id, get_e = False) for track_frame, intensity_frame in zip(track_frames, intensity_frames)]))
        if 'diameter' in features: 
            diameter.append(np.median([get_diameter_cellenONE(frame, track_id) for frame in track_frames]))
        if 'elongation' in features: 
            elongation.append(np.median([get_elongation_cellenONE(frame, track_id) for frame in track_frames]))
        if 'intensity_not_norm' in features: 
            intensity_not_norm.append(np.median([get_gray_intensity_cellenONE(frame = track_frame, intensity_image = intensity_frame, track_id = track_id) for track_frame, intensity_frame in zip(track_frames, intensity_frames)]))

        track_ids_to_save.append(track_id)

    if display_plot: 
        if 'area' in features:
            #print(f"The area is {area}")
            #print(f"The track ids are {track_ids}")
            histogram_with_best_fit(area, "Histogram of Object Area", 'area', output_directory, dataset = dataset)
        if 'length' in features:
            histogram_with_best_fit(length, "Histogram of Object length", 'length', output_directory, dataset = dataset)
        if 'width' in features:
            histogram_with_best_fit(width, "Histogram of Object width", 'width', output_directory, dataset = dataset)
        if 'circularity' in features: 
            histogram_with_best_fit(circularity, "Histogram of Object circularity", 'circularity', output_directory, dataset = dataset)
        if 'intensity' in features: 
            histogram_with_best_fit(intensity, "Histogram of Object intensity", 'intensity', output_directory, dataset = dataset)
        if 'energy' in features: 
            histogram_with_best_fit(energy, "Histogram of Object energy", 'energy', output_directory, dataset = dataset)
        if 'homogeneity' in features: 
            histogram_with_best_fit(homogeneity, "Histogram of Object homogeneity", 'homogeneity', output_directory, dataset = dataset)
        if 'diameter' in features: 
            histogram_with_best_fit(diameter, "Histogram of Object diameter", 'diameter', output_directory, dataset = dataset)
        if 'elongation' in features: 
            histogram_with_best_fit(elongation, "Histogram of Object elongation", 'elongation', output_directory, dataset = dataset)
        if 'intensity_not_norm' in features: 
            histogram_with_best_fit(intensity_not_norm, "Histogram of Object intensity non-normalized", 'intensity_not_norm', output_directory, dataset = dataset)
    
    return pd.DataFrame({'track_id': track_ids_to_save,
                        'area': area, 
                        'length': length, 
                        'width': width, 
                        'circularity': circularity, 
                        'intensity': intensity,
                        'energy': energy, 
                        'homogeneity': homogeneity,
                        'diameter': diameter, 
                        'elongation': elongation, 
                        'intensity_not_norm': intensity_not_norm, 
                        })

def feature_extraction_across_frames(track_info_file, tif_directory, png_directory, features = ['area', 'length', 'width', 'circularity', 'intensity', 'energy', 'homogeneity', 'diameter', 'elongation', 'intensity_not_norm'], track = 'all', track_ids = None, output_directory = None):

    # Load track information
    track_info = pd.read_csv(track_info_file, sep='\s+', header=None, names=['Track_ID', 'Start', 'End', 'Parent'], dtype={'Track_ID': int, 'Start': int, 'End': int, 'Parent': int})

    if track == 'all':
        track_ids = list(sorted(track_info['Track_ID']))
        if track_ids is None: raise RuntimeError("The track ID is None:(")


    for i in track_ids: 
        area = [] 
        length = []
        width  = []
        circularity = []
        intensity = []
        energy = []
        homogeneity = []
        diameter = []
        elongation = []
        intensity_not_norm = []
        
        track_id, start_frame, end_frame, parent_id = track_info.loc[track_info['Track_ID'] == i].values[0]

        frame_nums = []
        for frame in range(start_frame, end_frame+1):
            track_frame = tiff.imread(os.path.join(tif_directory, f'man_track{frame:04d}.tif'))
            intensity_frame = np.array(Image.open(os.path.join(png_directory, f'man_track{frame:04d}.png')).convert('L'))
            w =int((intensity_frame.shape[1]-10)/2)
            intensity_frames.append(intensity_frame[:,:w])
            #intensity_frame = np.array(Image.open(os.path.join(png_directory, f'man_track{frame:04d}.png')).convert('L'))[:,:192]

            if track_id not in np.unique(track_frame):
                continue
            frame_nums.append(frame)
            #print(f"input {frame} into frame nums {frame_nums}")
            if 'area' in features:
                #print(f"input {np.sum(track_frame == track_id)} into area {area}")
                area.append(np.sum(track_frame == track_id))
            if 'length' in features:
                length.append(get_h(track_frame, track_id))
            if 'width' in features:
                width.append(get_w(track_frame, track_id))
            if 'circularity' in features: 
                circularity.append(get_circ(track_frame, track_id))
            if 'intensity' in features: 
                intensity.append(get_intensity(frame = track_frame, intensity_image = intensity_frame, track_id = track_id))
            if 'energy' in features: 
                energy.append(get_glcm_features(frame = track_frame, intensity_image = intensity_frame, track_id = track_id, get_e = True))
            if 'homogeneity' in features: 
                homogeneity.append(get_glcm_features(frame = track_frame, intensity_image = intensity_frame, track_id = track_id, get_e = False))
            if 'diameter' in features:
                diameter.append(get_diameter_cellenONE(track_frame, track_id))
            if 'elongation' in features:
                elongation.append(get_elongation_cellenONE(track_frame, track_id))
            if 'intensity_not_norm' in features: 
                intensity_not_norm.append(get_gray_intensity_cellenONE(frame = track_frame, intensity_image = intensity_frame, track_id = track_id))
            


        if 'area' in features:
            #print(f"The area is {area} with the frame numbers {frame_nums}")
            linear_plot_with_summary(area, frame_nums, f'track#{track_id} area', xlab = "area", output_directory=output_directory)
        if 'length' in features:
            linear_plot_with_summary(length, frame_nums, f'track#{track_id} length', xlab = "length", output_directory=output_directory)
        if 'width' in features:
            linear_plot_with_summary(width, frame_nums, f'track#{track_id} width', xlab = "width", output_directory=output_directory)
        if 'circularity' in features: 
            linear_plot_with_summary(circularity, frame_nums, f'track#{track_id} circularity', xlab = "circularity", output_directory=output_directory)
        if 'intensity' in features: 
            linear_plot_with_summary(intensity, frame_nums, f'track#{track_id} intensity', xlab = "intensity", output_directory=output_directory)
        if 'energy' in features:
            linear_plot_with_summary(energy, frame_nums, f'track#{track_id} energy', xlab = "energy", output_directory=output_directory)
        if 'homogeneity' in features: 
            linear_plot_with_summary(homogeneity, frame_nums, f'track#{track_id} homogeneity', xlab = "homogeneity", output_directory=output_directory)
        if 'diameter' in features: 
            linear_plot_with_summary(diameter, frame_nums, f'track#{track_id} diameter', xlab = "diameter", output_directory=output_directory)
        if 'elongation' in features: 
            linear_plot_with_summary(elongation, frame_nums, f'track#{track_id} elongation', xlab = "elongation", output_directory=output_directory)
        if 'intensity_not_norm' in features: 
            linear_plot_with_summary(intensity_not_norm, frame_nums, f'track#{track_id} intensity_not_norm', xlab = "intensity_not_norm", output_directory=output_directory)


def feature_extraction_across_dataset(track_info_files, tif_directories, png_directories, dataset_names, features = ['area', 'length', 'width', 'circularity', 'intensity', 'energy', 'homogeneity', 'diameter', 'elongation', 'intensity_not_norm'], track = 'all', track_ids = None, output_directory = None, display_plot = True):
    if len(dataset_names) != len(track_info_files) or len(dataset_names) != len(tif_directories) or len(dataset_names) != len(png_directories):
        print(f"Error: Invalid Input: There are {len(dataset_names)} datasets but there are len(track_info_files) track info files, {len(tif_directories)} tif folders, and {png_directories} image folders.")
        sys.exit(1)
    
    area = [] 
    length = []
    width  = []
    circularity = []
    intensity = []
    energy = []
    homogeneity = []
    diameter = []
    elongation = []
    intensity_not_norm = []

    for i, name in enumerate(dataset_names):
        temp = feature_extraction_across_tracks(track_info_files[i], tif_directories[i], png_directories[i], features = features, output_directory = output_directory, display_plot = False)
        if 'area' in features:
            area.append(temp['area'])
        if 'length' in features:
            length.append(temp['length'])
        if 'width' in features:
            width.append(temp['width'])
        if 'circularity' in features: 
            circularity.append(temp['circularity'])
        if 'intensity' in features: 
            intensity.append(temp['intensity'])
        if 'energy' in features: 
            energy.append(temp['energy'])
        if 'homogeneity' in features: 
            homogeneity.append(temp['homogeneity'])
        if 'diameter' in features: 
            diameter.append(temp['diameter'])
        if 'elongation' in features: 
            elongation.append(temp['elongation'])
        if 'intensity_not_norm' in features: 
            intensity_not_norm.append(temp['intensity_not_norm'])
        


    if 'area' in features:
        create_violinplot_with_boxplot(area, xticklabels = dataset_names, title = 'area', output_directory=output_directory)
    if 'length' in features:
        create_violinplot_with_boxplot(length, xticklabels = dataset_names, title = 'length', output_directory=output_directory)
    if 'width' in features:
        create_violinplot_with_boxplot(width, xticklabels = dataset_names, title = 'width', output_directory=output_directory)
    if 'circularity' in features: 
        create_violinplot_with_boxplot(circularity, xticklabels = dataset_names, title = 'circularity', output_directory=output_directory)
    if 'intensity' in features: 
        create_violinplot_with_boxplot(intensity, xticklabels = dataset_names, title = 'intensity', output_directory=output_directory)
    if 'energy' in features: 
        create_violinplot_with_boxplot(energy, xticklabels = dataset_names, title = 'energy', output_directory=output_directory)
    if 'homogeneity' in features: 
        create_violinplot_with_boxplot(homogeneity, xticklabels = dataset_names, title = 'homogeneity', output_directory=output_directory)
    if 'diameter' in features: 
        create_violinplot_with_boxplot(diameter, xticklabels = dataset_names, title = 'diameter', output_directory=output_directory)
    if 'elongation' in features: 
        create_violinplot_with_boxplot(elongation, xticklabels = dataset_names, title = 'elongation', output_directory=output_directory)
    if 'intensity_not_norm' in features: 
        create_violinplot_with_boxplot(intensity_not_norm, xticklabels = dataset_names, title = 'intensity_not_norm', output_directory=output_directory)



## Histogram across objects 

In [ ]:
dataset_names = ["A138974A", "A146237A", "A118880", ]#"A138856A"]

track_info_files = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0/man_track_test.txt",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0/man_track.txt",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0/man_track.txt", 
    #"/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0/man_track.txt"
]
tif_directories = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0/",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0/",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0/", 
    #"/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0/"
]
png_directories = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0_imgs",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0_imgs",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0_imgs", 
    #"/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0_imgs"
]
output_directory = f'/projects/steiflab/scratch/leli/trackastra/feat_extract_analysis'

In [ ]:
features = feature_extraction_across_tracks(track_info_files[0], tif_directories[0], png_directories[0], output_directory = output_directory, dataset = "A138974A")
print(features)

features = feature_extraction_across_tracks(track_info_files[1], tif_directories[1], png_directories[1], output_directory = output_directory, dataset = "A146237A")
print(features)

features = feature_extraction_across_tracks(track_info_files[2], tif_directories[2], png_directories[2], output_directory = output_directory, dataset = "A118880")
print(features)

## Line plot across frames 

In [ ]:
feature_extraction_across_frames(track_info_file, tif_directory, png_directory, features = ['area',], track = 'not all', track_ids = [6, 217, 253, 133, 175, 142, 191, 258, 289, 271, 341, 380, 425], output_directory = output_directory)

feature_extraction_across_frames(track_info_file, tif_directory, png_directory, features = ['length',], track = 'not all', track_ids = [6, 217, 133, 175, 142, 184, 271, 253, 284, 289, 425], output_directory = output_directory)

feature_extraction_across_frames(track_info_file, tif_directory, png_directory, features = ['width',], track = 'not all', track_ids = [6, 39, 175, 142, 133, 217, 253, 289, 425, 326, 363], output_directory = output_directory)

feature_extraction_across_frames(track_info_file, tif_directory, png_directory, features = ['circularity',], track = 'not all', track_ids =  [38, 79, 39, 142, 205, 202, 231, 322, 39, 363, 217, 341], output_directory = output_directory)

feature_extraction_across_frames(track_info_file, tif_directory, png_directory, features = ['intensity',], track = 'not all', track_ids = [3, 184, 271, 284, 363, 380], output_directory = output_directory)

## Boxplot across dataset

In [ ]:
dataset_names = ["A138974A", "A146237A", "A118880", ]#"A138856A"]

track_info_files = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0/man_track_test.txt",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0", 
    #"/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0"
]
tif_directories = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0/",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0/",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0/", 
    #"/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0/"
]
png_directories = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0_imgs",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0_imgs",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0_imgs", 
    #"/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0_imgs"
]
output_directory = f'/projects/steiflab/scratch/leli/trackastra/feat_extract_analysis'

In [ ]:
feature_extraction_across_dataset(track_info_files, tif_directories, png_directories, dataset_names, output_directory = output_directory)

## Question: Can roundness distinguish between actual cells and debris

The approach we will take here is to randomly pick out 20 objects that we have high confidence in. To ensure class balance, there will be 10 labelled as debris and 10 labelled as cell. We will set up a threshold based on entropy and plot out the confusion matrix. 

In [ ]:

df = pd.read_csv('/projects/steiflab/scratch/leli/trackastra/track_to_well/track_to_well.csv')

# Function to get unique positive track IDs
def get_unique_non_zero_track_ids(df, type_value):
    track_counts = df[(df['type'] == type_value) & (df['track_ID'] > 0)]['track_ID'].value_counts()
    unique_track_ids = track_counts[track_counts == 1].index.tolist()
    return unique_track_ids, len(unique_track_ids), len(unique_track_ids) / len(track_counts) * 100

# Get the list of unique non-zero track IDs for livecell
livecell_unique_track_ids, livecell_unique_count, livecell_unique_percentage = get_unique_non_zero_track_ids(df, 'singlecell')

# Get the list of unique non-zero track IDs for debris
debris_unique_track_ids, debris_unique_count, debris_unique_percentage = get_unique_non_zero_track_ids(df, 'debris')

print("Unique non-zero track IDs for livecell:")
print(livecell_unique_track_ids)
print(f"Count: {livecell_unique_count}")
print(f"Percentage: {livecell_unique_percentage:.2f}%")

print("\nUnique non-zero track IDs for debris:")
print(debris_unique_track_ids)
print(f"Count: {debris_unique_count}")
print(f"Percentage: {debris_unique_percentage:.2f}%")

Here we prepare the features for live cells and debris

In [ ]:
dataset_names = ["A138974A", "A146237A", "A118880", ]#"A138856A"]

track_info_files = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0/man_track_test.txt",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_postprocessed_2.0/man_track.txt", 
    #"/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0/man_track.txt"
]
tif_directories = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0/",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0/",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_postprocessed_2.0/", 
    #"/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0/"
]
png_directories = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0_imgs",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0_imgs",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_postprocessed_2.0_imgs", 
    #"/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0_imgs"
]
output_directory = f'/projects/steiflab/scratch/leli/trackastra/feat_extract_analysis'

In [ ]:
livecell_feature = feature_extraction_across_tracks(track_info_files[0], tif_directories[0], png_directories[0], track = 'not all', track_ids=livecell_unique_track_ids, output_directory = None, display_plot = False, dataset = None)
debris_feature = feature_extraction_across_tracks(track_info_files[0], tif_directories[0], png_directories[0], track = 'not all', track_ids=debris_unique_track_ids, output_directory = None, display_plot = False, dataset = None)


let us see the results 

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt

# Assuming 'livecell_df' and 'debris_df' are your dataframes
livecell_df = pd.DataFrame({'circularity': livecell_feature['circularity']})
debris_df = pd.DataFrame({'circularity': debris_feature['circularity']})
print(livecell_df.shape)
print(debris_df.shape)
# Sample 60 random examples from each dataframe
livecell_sample = livecell_df.sample(n=20, random_state=42)
debris_sample = debris_df.sample(n=20, random_state=42)

# Combine these samples into a single dataframe
livecell_sample['label'] = 'livecell'
debris_sample['label'] = 'debris'
combined_df = pd.concat([livecell_sample, debris_sample])

# Prepare the data for the decision tree classifier
X = combined_df[['circularity']]
y = combined_df['label']

# Encode the labels as integers
y = y.map({'livecell': 0, 'debris': 1})

# Train a depth-1 decision tree (decision stump)
clf = DecisionTreeClassifier(max_depth=1, criterion='entropy')
clf.fit(X, y)

# Predict on the training data
y_pred = clf.predict(X)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)

# Generate classification metrics
report = classification_report(y, y_pred, target_names=['livecell', 'debris'], output_dict=True)
conf_matrix = confusion_matrix(y, y_pred)

# Add accuracy for each class
report['livecell']['accuracy'] = conf_matrix[0, 0] / conf_matrix[0].sum()
report['debris']['accuracy'] = conf_matrix[1, 1] / conf_matrix[1].sum()

# Display the accuracy
print(f"Overall Accuracy: {accuracy:.2f}")

# Display the classification report
print("\nClassification Report:")
for label, metrics in report.items():
    if label in ['livecell', 'debris']:
        print(f"{label.capitalize()}:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value:.2f}")
        print()

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=['livecell', 'debris'])
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

# Visualize the decision tree
plt.figure(figsize=(10, 6))
plot_tree(clf, feature_names=['circularity'], class_names=['livecell', 'debris'], filled=True, fontsize=20)
plt.title('Decision Tree using Entropy')
plt.show()


## Question: Can features extracted be clustered into 2 as shown in the violin plot

In [ ]:
dataset_names = ["A138974A", "A146237A", "A118880", "A138856A"]

track_info_files = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0/man_track_test.txt",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0/man_track.txt",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0/man_track.txt", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0/man_track.txt"
]
tif_directories = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0/",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0/",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0/", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0/"
]
png_directories = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0_imgs",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0_imgs",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0_imgs", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0_imgs"
]
output_directory = f'/projects/steiflab/scratch/leli/trackastra/feat_extract_analysis'

In [ ]:
siha_features = feature_extraction_across_tracks(track_info_files[3], tif_directories[3], png_directories[3], output_directory = None, dataset = "A138856A", display_plot = False, track = 'not all', track_ids = target_ids)

In [ ]:
import pandas as pd
import numpy as np
import umap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(siha_features[['area', 'length', 'width', 'circularity', 'intensity', 'energy', 'homogeneity']])

# Apply UMAP with different parameters
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='euclidean')
umap_embedding = umap_model.fit_transform(scaled_features)

# Plot UMAP embedding
plt.figure(figsize=(10, 8))
plt.scatter(umap_embedding[:, 0], 
                umap_embedding[:, 1])
plt.title('UMAP of Features')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.legend()
plt.show()

# Apply K-Means clustering
#kmeans = KMeans(n_clusters=2, random_state=42)
#clusters = kmeans.fit_predict(umap_embedding)
clusters = labs

# Add cluster labels to the siha_features DataFrame
siha_features['Cluster_n3'] = clusters

# Plot UMAP embedding
plt.figure(figsize=(10, 8))
for cluster in np.unique(clusters):
    plt.scatter(umap_embedding[clusters == cluster, 0], 
                umap_embedding[clusters == cluster, 1], 
                label=f'Cluster {cluster}', 
                s=10)
plt.title('UMAP of Features')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.legend()
plt.show()

'''# Apply K-Means clustering
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(umap_embedding)

# Add cluster labels to the siha_features DataFrame
siha_features['Cluster_n4'] = clusters

# Plot UMAP embedding
plt.figure(figsize=(10, 8))
for cluster in np.unique(clusters):
    plt.scatter(umap_embedding[clusters == cluster, 0], 
                umap_embedding[clusters == cluster, 1], 
                label=f'Cluster {cluster}', 
                s=10)
plt.title('UMAP of Features')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.legend()
plt.show()'''

siha_features


In [ ]:
import seaborn as sns

# Generate box plots for each feature based on the clusters
plt.figure(figsize=(15, 10))
for i, feature in enumerate(siha_features.columns[:-1]):  # Exclude the cluster column
    if feature == "track_id" or feature == "Cluster_n3": continue
    plt.subplot(2, 3, i)
    sns.boxplot(x='Cluster_n3', y=feature, data=siha_features)
    plt.title(f'Boxplot of {feature} by Cluster')
    plt.xlabel('Cluster')
    plt.ylabel(feature)
plt.tight_layout()
plt.show()

In [ ]:

# Generate box plots for each feature based on the clusters
plt.figure(figsize=(15, 10))
for i, feature in enumerate(siha_features.columns[:-1]):  # Exclude the cluster column
    if feature == "track_id" or feature == "Cluster_n3": continue
    plt.subplot(2, 3, i)
    sns.boxplot(x='Cluster_n4', y=feature, data=siha_features)
    plt.title(f'Boxplot of {feature} by Cluster')
    plt.xlabel('Cluster')
    plt.ylabel(feature)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# Display the value counts for each cluster
cluster_counts = siha_features['Cluster_n3'].value_counts()
print("Value counts for each cluster:")
print(cluster_counts)

# Randomly select 10 track IDs from each cluster
random_track_ids = {}
for cluster in cluster_counts.index:
    cluster_df = siha_features[siha_features['Cluster_n3'] == cluster]
    random_track_ids[cluster] = cluster_df['track_id'].sample(n=10, random_state=42).values

together = []
# Display the random track IDs for each cluster
for cluster, track_ids in random_track_ids.items():
    print(f"\nRandom track IDs from Cluster {cluster}:")
    print(sorted(track_ids))
    together.append(track_ids)

print(sorted([item for sublist in together for item in sublist]))

After manually going over all the 40 examples,

Cluster 0: single cell 

Cluster 1: debris + watermark

Cluster 2: watermark

Cluster 3: watermark

In [ ]:
print(np.median(siha_features.loc[siha_features['Cluster_n4'] != 0]['area']))
print(np.mean(siha_features.loc[siha_features['Cluster_n4'] != 0]['area']))
print(np.std(siha_features.loc[siha_features['Cluster_n4'] != 0]['area']))
print(np.max(siha_features.loc[siha_features['Cluster_n4'] != 0]['area']))
siha_features.loc[siha_features['Cluster_n4'] != 0]['area']

## Question: Can the features collected distinguish between cellenONE discarded and recorded cobjects

In [ ]:
dataset_names = ["A138974A", "A146237A", "A118880", "A138856A"]

track_info_files = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0/man_track_test.txt",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0/man_track.txt",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0/man_track.txt", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0/man_track.txt"
]
tif_directories = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0/",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0/",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0/", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0/"
]
png_directories = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0_imgs",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0_imgs",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0_imgs", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0_imgs"
]
output_directory = f'/projects/steiflab/scratch/leli/trackastra/feat_extract_analysis'

In [ ]:
a = [4, 13, 18, 21, 22, 70, 72, 80, 82, 93, 111, 113, 121, 130, 132, 133, 135, 136, 139, 145, 149, 151, 163, 180, 183, 185, 191, 208, 212, 231, 233, 239, 250, 251, 252, 254, 256, 259, 274, 281, 289, 291, 297, 299, 306, 318, 321, 329, 333, 341, 343, 348, 349, 350, 351, 358, 360, 372, 379, 389, 401, 422, 433, 455, 456, 463, 479, 480, 486, 495, 496, 507, 512, 513, 522, 526, 531, 538, 539, 540, 545, 548, 557, 565, 575, 587, 589, 591, 596, 605, 616, 628, 633, 636, 638, 647, 652, 674, 678, 683, 686, 693, 710, 715, 723, 736, 737, 747, 755, 757, 773, 774, 775, 779, 792, 805, 807, 808, 809, 814, 820, 829, 849, 851, 858, 860, 872, 875, 881, 901, 920, 933, 959, 961, 966, 975, 985, 989, 1008, 1009, 1011, 1013, 1025, 1032, 1035, 1044, 1048, 1064, 1065, 1072, 1077, 1091, 1092, 1093, 1095, 1097, 1101, 1107, 1118, 1126, 1129, 1130, 1131, 1144, 1154, 1167, 1171, 1175, 1184, 1187, 1200, 1207, 1212, 1223, 1226, 1232, 1237, 1240, 1242, 1243, 1257, 1260, 1271, 1272, 1302, 1305, 1314, 1317, 1334, 1335, 1337, 1341, 1346, 1356, 1360, 1365, 1374, 1377, 1386, 1387, 1388, 1394, 1397, 1439, 1440, 1449, 1461, 1466, 1478, 1503, 1515, 1518, 1522, 1527, 1531, 1539, 1555, 1556, 1558, 1568, 1573, 1577, 1591, 1594, 1595, 1596, 1609, 1611, 1612, 1621, 1623, 1625, 1627, 1632, 1634, 1644, 1645, 1655, 1658, 1659, 1664, 1673, 1684, 1688, 1690, 1695, 1701, 1705, 1711, 1713, 1714, 1715, 1717, 1726, 1731, 1732, 1735, 1744, 1748, 1757, 1759, 1766, 1784, 1791, 1802, 1804, 1805, 1810, 1813, 1829, 1855, 1857, 1863, 1866, 1867, 1871, 1887, 1905, 1906, 1908, 1926, 1927, 1932, 1949, 1950, 1952, 1959, 1964, 1970, 1973, 1981, 1985, 1998, 2000, 2017, 2024, 2027, 2029, 2031, 2040, 2045, 2051, 2054, 2062, 2064, 2070, 2079, 2080, 2084, 2092, 2095, 2102, 2106, 2128, 2138, 2142, 2147, 2148, 2150, 2162, 2174, 2177, 2180, 2188, 2189, 2190, 2195, 2196, 2213, 2227, 2228, 2231, 2241, 2242, 2245, 2257, 2260, 2287, 2294, 2299, 2307, 2308, 2310, 2325, 2338, 2339, 2343, 2354, 2360, 2365, 2369, 2379, 2400, 2414, 2422, 2423, 2444]
b = [1, 6, 7, 10, 12, 14, 20, 23, 25, 29, 30, 31, 32, 33, 34, 39, 43, 45, 47, 50, 54, 60, 63, 73, 74, 75, 79, 83, 84, 86, 94, 105, 108, 116, 122, 123, 144, 150, 152, 154, 157, 162, 164, 171, 179, 187, 192, 195, 198, 199, 211, 214, 219, 220, 224, 225, 227, 228, 230, 232, 236, 243, 246, 253, 255, 260, 262, 271, 275, 276, 278, 280, 288, 292, 294, 298, 302, 303, 312, 316, 317, 325, 328, 330, 338, 352, 354, 355, 356, 357, 359, 361, 366, 371, 374, 376, 377, 378, 384, 385, 399, 400, 402, 403, 405, 406, 407, 409, 412, 414, 415, 416, 417, 418, 419, 421, 431, 432, 441, 450, 451, 452, 454, 457, 458, 460, 464, 465, 467, 469, 472, 473, 478, 481, 490, 491, 492, 493, 498, 499, 500, 501, 502, 506, 508, 509, 510, 511, 514, 515, 517, 523, 527, 528, 536, 537, 541, 542, 559, 560, 566, 569, 570, 572, 573, 574, 577, 578, 583, 586, 593, 594, 604, 607, 609, 613, 615, 617, 631, 632, 635, 639, 648, 653, 654, 656, 659, 661, 667, 681, 682, 684, 685, 687, 688, 694, 695, 699, 706, 707, 708, 709, 711, 713, 714, 719, 721, 729, 730, 731, 743, 744, 753, 754, 758, 759, 762, 767, 768, 769, 770, 780, 781, 787, 788, 794, 798, 803, 804, 806, 815, 818, 821, 822, 823, 835, 837, 839, 842, 843, 845, 846, 850, 853, 854, 855, 862, 863, 864, 868, 870, 871, 873, 874, 876, 877, 879, 880, 884, 886, 888, 889, 890, 891, 892, 894, 896, 898, 899, 913, 916, 921, 925, 928, 929, 934, 936, 937, 944, 945, 950, 951, 952, 953, 954, 957, 960, 962, 963, 967, 968, 970, 977, 982, 983, 984, 986, 990, 992, 999, 1007, 1014, 1018, 1019, 1020, 1026, 1027, 1028, 1031, 1037, 1038, 1039, 1040, 1041, 1046, 1047, 1055, 1056, 1057, 1058, 1060, 1061, 1070, 1075, 1076, 1085, 1089, 1090, 1094, 1096, 1103, 1104, 1110, 1111, 1115, 1117, 1123, 1128, 1132, 1134, 1139, 1141, 1142, 1143, 1145, 1147, 1148, 1149, 1150, 1155, 1163, 1168, 1172, 1174, 1176, 1179, 1180, 1183, 1185, 1186, 1190, 1194, 1199, 1201, 1204, 1205, 1209, 1211, 1218, 1219, 1220, 1221, 1222, 1224, 1227, 1228, 1247, 1251, 1256, 1258, 1265, 1277, 1278, 1279, 1283, 1295, 1296, 1303, 1306, 1308, 1311, 1312, 1313, 1332, 1333, 1352, 1355, 1362, 1364, 1367, 1368, 1370, 1371, 1372, 1373, 1375, 1376, 1378, 1380, 1381, 1382, 1383, 1385, 1390, 1391, 1392, 1393, 1396, 1398, 1400, 1404, 1417, 1419, 1421, 1422, 1425, 1426, 1430, 1431, 1432, 1435, 1441, 1444, 1445, 1451, 1454, 1459, 1460, 1467, 1468, 1469, 1471, 1472, 1473, 1479, 1480, 1481, 1482, 1483, 1493, 1494, 1496, 1497, 1499, 1504, 1506, 1507, 1512, 1513, 1514, 1519, 1520, 1523, 1524, 1525, 1540, 1541, 1545, 1546, 1547, 1553, 1554, 1557, 1561, 1569, 1570, 1571, 1579, 1582, 1583, 1592, 1593, 1608, 1610, 1615, 1616, 1618, 1619, 1620, 1628, 1635, 1638, 1650, 1652, 1654, 1656, 1660, 1663, 1672, 1675, 1687, 1689, 1698, 1699, 1702, 1703, 1704, 1709, 1712, 1718, 1719, 1724, 1725, 1727, 1730, 1733, 1734, 1736, 1738, 1739, 1742, 1743, 1745, 1747, 1754, 1755, 1758, 1765, 1774, 1782, 1783, 1785, 1787, 1788, 1792, 1795, 1796, 1797, 1798, 1799, 1801, 1803, 1806, 1808, 1809, 1811, 1812, 1815, 1816, 1822, 1824, 1828, 1830, 1832, 1833, 1835, 1843, 1844, 1845, 1851, 1852, 1854, 1859, 1860, 1868, 1870, 1876, 1877, 1881, 1882, 1883, 1885, 1886, 1889, 1890, 1895, 1901, 1907, 1911, 1913, 1918, 1919, 1920, 1923, 1924, 1925, 1930, 1931, 1940, 1943, 1947, 1948, 1951, 1953, 1954, 1956, 1958, 1963, 1968, 1969, 1972, 1976, 1977, 1978, 1979, 1986, 1991, 1995, 2002, 2006, 2009, 2010, 2014, 2015, 2018, 2019, 2028, 2034, 2043, 2046, 2052, 2053, 2057, 2059, 2060, 2063, 2065, 2067, 2068, 2071, 2072, 2073, 2075, 2077, 2078, 2081, 2086, 2087, 2090, 2091, 2093, 2094, 2096, 2098, 2099, 2104, 2105, 2107, 2110, 2111, 2112, 2113, 2114, 2115, 2120, 2124, 2125, 2127, 2131, 2134, 2137, 2139, 2140, 2143, 2144, 2149, 2151, 2153, 2158, 2159, 2160, 2163, 2165, 2166, 2167, 2168, 2169, 2171, 2173, 2175, 2176, 2178, 2179, 2186, 2191, 2192, 2194, 2197, 2198, 2199, 2201, 2202, 2204, 2206, 2208, 2209, 2210, 2212, 2214, 2218, 2219, 2222, 2224, 2230, 2235, 2236, 2237, 2240, 2246, 2248, 2249, 2250, 2254, 2256, 2268, 2270, 2272, 2275, 2278, 2280, 2281, 2284, 2292, 2293, 2295, 2297, 2304, 2305, 2306, 2313, 2318, 2324, 2330, 2337, 2340, 2342, 2344, 2345, 2348, 2350, 2351, 2355, 2356, 2370, 2372, 2381, 2387, 2395, 2396, 2397, 2399, 2401, 2405, 2406, 2407, 2408, 2412, 2416, 2427, 2428, 2429, 2432, 2433, 2434, 2436]
labs = [0 for i in range(len(a))] + [1 for i in range(len(b))]
target_ids = a + b

print(f"There are {len(a)} isolated and {len((b))} discarded and 20% of discarded is {0.2 * len(b)}")

In [ ]:
siha_features = feature_extraction_across_tracks(track_info_files[3], tif_directories[3], png_directories[3], output_directory = None, dataset = "A138856A", display_plot = False, track = 'not all', track_ids = target_ids)
siha_features['label'] = ['isolated' for i in range(len(a))] + ['discarded' for i in range(len(b))]

siha_features.to_csv("/projects/steiflab/scratch/leli/A138856A/htert_20230822_131349_843.Run/features.csv", index = False)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming siha_features is your DataFrame and labs is your list of labels
siha_features['label'] = labs

# Determine the minimum class size
min_class_size = siha_features['label'].value_counts().min()

# Sample the same number of examples from each class
balanced_data = siha_features.groupby('label').sample(n=min_class_size, random_state=42)

# List of columns to plot
columns_to_plot = ['area', 'length', 'width', 'circularity', 'intensity', 'energy', 'homogeneity', 'diameter', 'elongation', 'intensity_not_norm']

# Loop through each column and create a violin plot
for column in columns_to_plot:
    plt.figure(figsize=(8, 6))
    sns.violinplot(x='label', y=column, data=balanced_data)
    plt.title(f'Violin Plot of {column.capitalize()} by Label')
    plt.xlabel('Cell Status')
    plt.ylabel(column.capitalize())
    plt.xticks(ticks=[0, 1], labels=['Isolated', 'Discarded'])
    plt.show()


In [ ]:
from scipy.stats import mannwhitneyu

# List of features to test
features_to_test = ['area', 'length', 'width', 'circularity', 'intensity', 'energy', 'homogeneity', 'diameter', 'elongation', 'intensity_not_norm']

# Split the data into the two groups
isolated = siha_features[siha_features['label'] == 0]
discarded = siha_features[siha_features['label'] == 1]

# Perform the Mann-Whitney U test for each feature
results = {}
for feature in features_to_test:
    stat, p_value = mannwhitneyu(isolated[feature], discarded[feature])
    results[feature] = {'U-statistic': stat, 'p-value': p_value}

# Print the results
for feature, result in results.items():
    print(f"Feature: {feature}")
    print(f"  U-statistic: {result['U-statistic']}")
    print(f"  p-value: {result['p-value']}\n")


In [ ]:
import pandas as pd

# Assume you have your labels in a list called 'labs' and your features in a dataframe 'siha_features'

# Create a DataFrame combining the labels and the features
data = pd.DataFrame({'Label': labs})
data = pd.concat([data, siha_features], axis=1)

# Group the data by label and calculate the summary statistics
summary_stats = data.groupby('Label').agg(['mean', 'median', 'std', 'min', 'max'])

# Print summary statistics for each feature
for feature in siha_features[['area', 'length', 'width', 'circularity', 'intensity', 'energy', 'homogeneity', 'diameter', 'elongation', 'intensity_not_norm']].columns:
    print(f"Summary Statistics for {feature}:")
    print(summary_stats[feature])
    print("\n")


In [ ]:
import numpy as np

def cohen_d(x, y):
    """Calculate Cohen's d."""
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    pooled_std = np.sqrt(((nx - 1) * np.std(x, ddof=1) ** 2 + (ny - 1) * np.std(y, ddof=1) ** 2) / dof)
    d = np.abs(np.mean(x) - np.mean(y)) / pooled_std
    return d

# Calculate Cohen's d for each feature
for feature in siha_features[['area', 'length', 'width', 'circularity', 'intensity', 'energy', 'homogeneity', 'diameter', 'elongation', 'intensity_not_norm']].columns:
    group_0 = data[data['Label'] == 0][feature]
    group_1 = data[data['Label'] == 1][feature]
    d = cohen_d(group_0, group_1)
    print(f"Cohen's d for {feature}: {d}")


Summary:
The effect sizes (Cohen's d) suggest that homogeneity is the most distinct feature between the two groups, with a medium effect size (0.50). This suggests that homogeneity could potentially be a useful feature in distinguishing between the "Isolated" and "Discarded" classes.
Features like circularity and width also show some level of differentiation but to a lesser extent.
The other features (area, length, intensity, energy) exhibit small effect sizes, indicating minimal differences between the two classes.
Implications:
While the p-values from the statistical tests indicated significant differences for most of the features, the effect sizes suggest that the magnitude of these differences is generally small. This could explain why the violin plots appear similar even though the p-values are low. The statistical significance (p-values) might be driven by the large sample size rather than large differences between the groups.

This analysis indicates that while there might be statistically significant differences between the groups, these differences might not be large enough to be practically meaningful.

In [ ]:
import pandas as pd
import numpy as np
import umap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(siha_features[['area', 'length', 'width', 'circularity', 'intensity', 'energy', 'homogeneity', 'diameter', 'elongation', 'intensity_not_norm']])

# Apply UMAP with different parameters
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='euclidean')
umap_embedding = umap_model.fit_transform(scaled_features)

# Plot UMAP embedding
plt.figure(figsize=(10, 8))
plt.scatter(umap_embedding[:, 0], 
                umap_embedding[:, 1])
plt.title('UMAP of Features')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.legend()
plt.show()

# Apply K-Means clustering
#kmeans = KMeans(n_clusters=2, random_state=42)
#clusters = kmeans.fit_predict(umap_embedding)
clusters = labs

# Plot UMAP embedding
plt.figure(figsize=(10, 8))
for cluster in np.unique(clusters):
    plt.scatter(umap_embedding[clusters == cluster, 0], 
                umap_embedding[clusters == cluster, 1], 
                label=f'Cluster {cluster}', 
                s=10)
plt.title('UMAP of Features')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.legend()
plt.show()

'''# Apply K-Means clustering
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(umap_embedding)

# Add cluster labels to the siha_features DataFrame
siha_features['Cluster_n4'] = clusters

# Plot UMAP embedding
plt.figure(figsize=(10, 8))
for cluster in np.unique(clusters):
    plt.scatter(umap_embedding[clusters == cluster, 0], 
                umap_embedding[clusters == cluster, 1], 
                label=f'Cluster {cluster}', 
                s=10)
plt.title('UMAP of Features')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.legend()
plt.show()'''

siha_features


## Question: Are all the cellenONE things we measure accurate? 

In [ ]:
import pandas as pd

# Load the CSV files into DataFrames
features = pd.read_csv('/projects/steiflab/scratch/leli/A138856A/htert_20230822_131349_843.Run/features.csv')
track_to_well = pd.read_csv('/projects/steiflab/scratch/leli/A138856A/htert_20230822_131349_843.Run/track_to_well/track_to_well.csv')
record = pd.read_csv('/projects/steiflab/scratch/leli/A138856A/htert_20230822_131349_843.Run/record.csv')

# Merge features and track_to_well on track_id and track_ID
merged_df = pd.merge(features, track_to_well, left_on='track_id', right_on='track_ID', how='inner')
if merged_df['track_id'].equals(merged_df['track_ID']):
    merged_df = merged_df.drop(columns=['track_ID'])
else:
    print("Warning: track_id and track_ID columns are not identical.")
    
import re

# Function to extract R and C values from the given pattern
def extract_R_C_values(filename):
    match = re.search(r'R(\d+)_C(\d+)', filename)
    if match:
        return int(match.group(1)), int(match.group(2))
    else:
        return None, None

# Apply the extraction function to the columns
merged_df[['R', 'C']] = merged_df['fluro_image'].apply(lambda x: pd.Series(extract_R_C_values(x)))
record[['R', 'C']] = record['ImageFile'].apply(lambda x: pd.Series(extract_R_C_values(x)))

# Merge the DataFrames based on R and C values
final_metadata_df = pd.merge(merged_df, record, on=['R', 'C'], how='inner')

# Step 4: Add the "tracked_" prefix to specific columns
columns_to_prefix = [
    'area', 'length', 'width', 'circularity', 'intensity',
    'energy', 'homogeneity', 'diameter', 'elongation', 
    'intensity_not_norm', 'label'
]

final_metadata_df = final_metadata_df.rename(
    columns={col: f"tracked_{col}" for col in columns_to_prefix}
)

# Step 5: Capitalize the first letter of all column names
final_metadata_df.columns = [col.capitalize() for col in final_metadata_df.columns]

# Save the final DataFrame to a CSV file
final_metadata_df.to_csv('metadata.csv', index=False)

# Save the final metadata DataFrame to a CSV file
final_metadata_df.to_csv('/projects/steiflab/scratch/leli/A138856A/htert_20230822_131349_843.Run/isolated_metadata.csv', index=False)

print("Metadata DataFrame saved as 'metadata.csv'")

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon
import matplotlib.pyplot as plt
import seaborn as sns

# Load the DataFrame (replace 'metadata.csv' with your actual file if needed)
df = pd.read_csv('/projects/steiflab/scratch/leli/A138856A/htert_20230822_131349_843.Run/isolated_metadata.csv')

# List of pairs to compare
pairs_to_compare = [
    ('Tracked_diameter', 'Diameter'),
    ('Tracked_elongation', 'Elongation'),
    ('Tracked_intensity_not_norm', 'Intensity')
]

# Wilcoxon signed-rank test results
wilcoxon_results = {}

for tracked_col, col in pairs_to_compare:
    stat, p_value = wilcoxon(df[tracked_col], df[col])
    wilcoxon_results[tracked_col] = (stat, p_value)

# Display Wilcoxon test results
for tracked_col, (stat, p_value) in wilcoxon_results.items():
    print(f"Wilcoxon signed-rank test for {tracked_col} vs. its pair:")
    print(f"  Test Statistic: {stat}, p-value: {p_value}\n")

# Plotting side-by-side boxplots for each pair
for tracked_col, col in pairs_to_compare:
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df[[tracked_col, col]])
    plt.title(f"Comparison of {tracked_col} and {col}")
    plt.ylabel("Values")
    plt.grid(True)
    plt.show()


In [ ]:
df[['Track_id', 'Last_tracked_frame', 'Last_tracked_tif', 'Imagefile', 'After_dispense_frame']].sample(n=10, random_state=42)
 


## Question: Is there systematic bias between cellenONE and Isolatrix predictions when it comes to isolating and discarding cells in terms of morphological features?

In [ ]:
import pandas as pd 
cellenONE_metadata = pd.read_csv("/projects/steiflab/scratch/leli/A138856A/htert_20230822_131349_843.Run/features.csv")

cellenONE_metadata['label'].value_counts()


Here we have for cellenONE isolated is 357 and for discarded 765. The next step is to get these ones for the isiolatrix

In [ ]:
dataset_names = ["A138974A", "A146237A", "A118880", "A138856A"]
printruns = ["5dropRun1", "5dropRun2", "5dropRun3", "5dropRun4", "5dropRun5", "10dropRun1", "10dropRun2", "10dropRun3", "10dropRun4"]

track_info_files = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0/man_track_test.txt",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0/man_track.txt",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0/man_track.txt", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0/man_track.txt", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun1/tracked_again_postprocessed_2.0/man_track.txt", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun2/tracked_again_postprocessed_2.0/man_track.txt", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun3/tracked_again_postprocessed_2.0/man_track.txt", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun4/tracked_again_postprocessed_2.0/man_track.txt", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun5/tracked_again_postprocessed_2.0/man_track.txt", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/10dropRun1/tracked_again_postprocessed_2.0/man_track.txt", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/10dropRun2/tracked_again_postprocessed_2.0/man_track.txt", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/10dropRun3/tracked_again_postprocessed_2.0/man_track.txt", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/10dropRun4/tracked_again_postprocessed_2.0/man_track.txt", 
]
tif_directories = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0/",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0/",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0/", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0/", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun1/tracked_again_postprocessed_2.0", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun2/tracked_again_postprocessed_2.0", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun3/tracked_again_postprocessed_2.0", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun4/tracked_again_postprocessed_2.0", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun5/tracked_again_postprocessed_2.0", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/10dropRun1/tracked_again_postprocessed_2.0", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/10dropRun2/tracked_again_postprocessed_2.0", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/10dropRun3/tracked_again_postprocessed_2.0", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/10dropRun4/tracked_again_postprocessed_2.0", 
]
png_directories = [
    "/projects/steiflab/scratch/leli/trackastra/A138974A/PrintRun_Apr1223_1311/tracked_postprocessed_2.0_imgs",
    "/projects/steiflab/scratch/leli/trackastra/A146237A/PrintRun_Mar2824_1524/tracked_postprocessed_2.0_imgs",
    "/projects/steiflab/scratch/leli/trackastra/A118880/PrintRun_Jan2624_1252/tracked_again_postprocessed_2.0_imgs", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0_imgs", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun1/tracked_again_postprocessed_2.0_imgs", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun2/tracked_again_postprocessed_2.0_imgs", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun3/tracked_again_postprocessed_2.0_imgs", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun4/tracked_again_postprocessed_2.0_imgs", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/5dropRun5/tracked_again_postprocessed_2.0_imgs", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/10dropRun1/tracked_again_postprocessed_2.0_imgs", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/10dropRun2/tracked_again_postprocessed_2.0_imgs", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/10dropRun3/tracked_again_postprocessed_2.0_imgs", 
    "/projects/steiflab/scratch/leli/trackastra/A138856A/10dropRun4/tracked_again_postprocessed_2.0_imgs", 

]
output_directory = f'/projects/steiflab/scratch/leli/trackastra/feat_extract_analysis'

In [ ]:
import pandas as pd 
import numpy as np 
# Step 1: Read the CSV files into separate DataFrames
dataframes = [pd.read_csv(f"/projects/steiflab/scratch/leli/A138856A/{pr}/track_to_well/track_to_well.csv") for pr in printruns]

# Step 2: Add a 'source' column to track the origin of each row
for i, df in enumerate(dataframes):
    df['source'] = printruns[i]

# Step 3: Concatenate all DataFrames into one large DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Step 4: Find and remove rows with duplicate fluro_image values
unique_df = combined_df.drop_duplicates(subset='fluro_image', keep=False)

# Step 5: Group track_IDs by Prediction and source
track_id_grouped = unique_df.groupby(['prediction', 'source'])['track_ID'].apply(list).to_dict()

# Step 6: Print the lists and their lengths
for (prediction, source), track_ids in track_id_grouped.items():
    temp = list(np.unique([t for t in track_ids if t > 0]))
    print(f"Prediction: {prediction}, Source: {source}, Count: {len(temp)}")
    
    print(f"Track IDs: {temp}")
    print()  # Add a new line for better readability


### Singlecell Isolatirx

In [ ]:
import sys
track_ids_dict = {
    "10dropRun1": [3, 6, 9, 11, 15, 18, 25, 30, 38],
    "10dropRun2": [1, 2, 3, 5, 13, 14, 15, 16, 17, 29, 30, 33, 34, 39, 41, 43, 44, 45, 49, 50, 51, 54, 55, 57, 59, 61, 63, 65, 66, 69, 72, 73, 74, 75, 78, 82, 83, 84, 86, 92, 99, 102, 103, 104, 106, 108, 109, 110, 113, 120, 121, 124, 125, 126, 129, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 150, 151, 152, 154, 155, 156, 158, 159, 160, 162, 169, 173, 178, 193, 196, 198, 207, 208, 209, 210, 212, 213, 215, 224, 225, 226, 234, 237, 238, 241, 244, 247],     "10dropRun3": [1, 4, 5, 6, 14, 18, 22, 30, 34, 35, 41, 46, 49, 59, 63, 99, 109, 115, 116, 121, 134, 144, 154, 160, 161, 173, 196, 200, 208, 212, 246, 285, 301, 326, 328, 360, 365, 367, 372, 402, 403, 429, 431, 434, 435, 436, 437, 438, 440, 445, 449, 452, 453, 459, 469, 491, 501, 507, 519, 520, 525, 535, 537, 539, 541],
    "10dropRun3": [1, 4, 5, 7, 11, 14, 18, 29, 33, 38, 41, 49, 51, 76, 80, 81, 93, 99, 105, 109, 110, 119, 144, 146, 156, 165, 186, 190, 215, 217, 235, 240, 242, 245, 261, 262, 278, 280, 283, 284, 285, 286, 287, 288, 293, 297, 300, 301, 306, 310, 316, 335, 340, 352, 358, 369, 370, 375, 381, 386, 391, 393], 
    "10dropRun4": [1, 10, 11, 15, 20, 22, 23, 24, 26, 29, 34, 36, 37, 39, 40, 41, 42, 45, 46, 47, 48, 49, 51, 53, 58, 59, 61, 65, 67, 70, 72, 73, 75, 76, 77, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 96, 99, 100, 101, 103, 105, 106, 107, 108, 109, 111, 112, 114, 115, 116, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 130, 131, 133, 136, 139, 148, 169, 171, 176, 179, 182, 183, 185, 190, 193, 195], 
    "5dropRun1": [1, 2, 3, 4, 5],
    "5dropRun2": [7, 8, 9, 10, 13, 15, 16, 19, 20, 21, 22, 23, 27, 28, 31, 32, 33, 34, 35, 37, 38, 39, 42, 43, 44, 46, 48, 55, 56, 58, 59, 64, 66, 67, 69, 70, 71, 73, 74, 85, 88, 89, 104, 105, 106, 109, 110, 114, 115, 116, 117, 118, 119, 120, 123, 124, 125, 126, 148, 150, 154, 161, 162, 163, 170, 171, 172, 174, 175, 176, 178, 180, 183, 185, 187, 188, 200, 203, 204, 206, 207, 211, 215, 237, 244, 245, 249, 257, 271, 295, 322, 323], 
    "5dropRun3": [2, 3, 4, 7, 10, 11, 15, 16, 17, 19, 23, 24, 25, 27, 32, 33, 36, 39, 41, 47, 55, 63, 68, 69, 71, 72, 75, 76, 78, 79, 80, 82, 83, 84, 87, 88, 92, 93, 97, 99, 100, 118, 124, 126, 127, 130, 131, 134, 140, 143, 145, 146, 151, 154, 155, 156, 157, 158, 159, 162, 164, 167, 169, 170, 173, 174, 176, 177, 180, 181, 182, 183, 185, 188, 189, 192, 193, 194, 195, 196, 200, 201, 204, 205, 206, 207, 208, 209, 210, 212, 221, 224, 229, 236, 239, 248, 260, 264, 266, 272], 
    "5dropRun4": [1, 2, 3, 4, 9, 12, 13, 14, 15, 16, 18, 19, 23, 27, 28, 30, 34, 35, 37, 39, 40, 42, 44, 45, 52, 56, 59, 62, 63, 76, 77, 79, 82, 83, 85, 87, 88, 91, 92, 94, 97, 98, 100, 106, 110, 111, 114, 115, 117, 118, 119, 121, 123, 124, 127, 133, 134, 136, 137, 144, 152, 154, 156, 157, 158, 159, 177, 180, 181, 183, 191, 202, 203, 214, 220, 221, 226, 229, 232, 241, 243, 244, 248, 256, 261, 263, 299], 
    "5dropRun5": [2, 4, 5, 10, 11, 15, 19, 31, 32, 40, 47, 55], 
}

total_track_num = 0
for run_name in track_ids_dict: 
    track_ids = track_ids_dict[run_name]
    print(f"{run_name} has {len(track_ids)}")
    total_track_num += len(track_ids)
print(f"In total there are {total_track_num} tracks ")
sys.exit()

for run_name in track_ids_dict:
    run_index = printruns.index(run_name) + 4  # Adjusting index to match the track_info_files and directory lists
    track_ids = track_ids_dict[run_name]
    print(f"this interation has {track_info_files[run_index]} and {track_ids}")
    print(f"The outdirectory should be ")
    temp = feature_extraction_across_tracks(
        track_info_files[run_index], 
        tif_directories[run_index], 
        png_directories[run_index], 
        output_directory=None, 
        dataset="A138856A", 
        display_plot=False, 
        track='not all', 
        track_ids=track_ids
    )
    temp.to_csv(f"/projects/steiflab/scratch/leli/A138856A/{run_name}/features_p1.csv", index = False)



### Non-singlecell Isolatrix 
Note cellenONE discard anything that is not a singlecell, to be fair, we shoud include both prediction 0 and 2 then.  


In [ ]:
track_ids_dict = {
    "10dropRun1": [2, 22],
    "10dropRun2": [21, 52, 53, 131, 161, 165],
    "10dropRun3": [13, 46, 87, 90, 106, 121, 136, 172, 208, 249, 263, 264, 309, 331, 333, 390],
    "10dropRun4": [2, 38, 81, 187],
    "5dropRun1": [],
    "5dropRun2": [2, 36, 61, 72, 90, 91, 127, 144, 155, 156, 179, 182, 191, 197, 199, 208, 212, 240, 242, 276, 288],
    "5dropRun3": [8, 37, 43, 45, 56, 101, 120, 160, 165, 191, 211, 243],
    "5dropRun4": [17, 49, 54, 55, 60, 67, 135, 166, 172, 186, 196, 199, 271, 282, 295],
    "5dropRun5": [14, 18, 27, 34, 42, 54], 
}
total_track_num = 0
for run_name in track_ids_dict: 
    track_ids = track_ids_dict[run_name]
    print(f"{run_name} has {len(track_ids)}")
    total_track_num += len(track_ids)
print(f"In total there are {total_track_num} tracks ")

for run_name in track_ids_dict:
    if not track_ids_dict[run_name]:
        continue  # Skip if the list is empty
    
    run_index = printruns.index(run_name) + 4  # Adjusting index to match the track_info_files and directory lists
    track_ids = track_ids_dict[run_name]
    print(f"this interation has {track_info_files[run_index]} and {track_ids}")
    feature_extraction_across_tracks(
        track_info_files[run_index], 
        tif_directories[run_index], 
        png_directories[run_index], 
        output_directory=output_directory, 
        dataset="A138856A", 
        display_plot=False, 
        track='not all', 
        track_ids=track_ids
    )
    temp.to_csv(f"/projects/steiflab/scratch/leli/A138856A/{run_name}/features_p2.csv", index = False)

In [ ]:
track_ids_dict = {
    "10dropRun1": [],
    "10dropRun2": [93, 148, 166, 235],
    "10dropRun3": [39, 64, 79, 85, 104, 137, 205, 299, 307, 376, 384, 387],
    "10dropRun4": [35, 50, 60, 98, 102, 144, 189],
    "5dropRun1": [],
    "5dropRun2": [5, 18, 40, 41, 53, 147, 151, 173, 186, 190, 263, 267, 317],
    "5dropRun3": [5, 54, 94, 98, 129, 161, 166, 250],
    "5dropRun4": [46, 61, 64, 73, 81, 96, 105, 163, 185, 219, 227],
    "5dropRun5": [66]
}

total_track_num = 0
for run_name in track_ids_dict: 
    track_ids = track_ids_dict[run_name]
    print(f"{run_name} has {len(track_ids)}")
    total_track_num += len(track_ids)
print(f"In total there are {total_track_num} tracks ")

for run_name in track_ids_dict:
    if not track_ids_dict[run_name]:
        continue  # Skip if the list is empty
    
    run_index = printruns.index(run_name) + 4  # Adjusting index to match the track_info_files and directory lists
    track_ids = track_ids_dict[run_name]
    print(f"this interation has {track_info_files[run_index]} and {track_ids}")
    feature_extraction_across_tracks(
        track_info_files[run_index], 
        tif_directories[run_index], 
        png_directories[run_index], 
        output_directory=output_directory, 
        dataset="A138856A", 
        display_plot=False, 
        track='not all', 
        track_ids=track_ids
    )
    temp.to_csv(f"/projects/steiflab/scratch/leli/A138856A/{run_name}/features_p0.csv", index = False)

## Gathering features for postprocessed track_to_well_pp.csv into one for the isolatrix paper

In [ ]:
import os
import pandas as pd

# List of folder names
printruns = ["htert_20230822_131349_843.Run", "5dropRun1", "5dropRun2", "5dropRun3", "5dropRun4", "5dropRun5", "10dropRun1", "10dropRun2", "10dropRun3", "10dropRun4"]

# Base directory path (replace with your actual path)
base_directory = "/projects/steiflab/scratch/leli/A138856A"
feature_metadata = pd.DataFrame()
# Iterate through each folder name
for printrun in printruns:
    track_to_well_path = os.path.join(base_directory, printrun, "track_to_well", "track_to_well_pp.csv")
    
    if os.path.exists(track_to_well_path):
        # Read the CSV file
        df = pd.read_csv(track_to_well_path)
        
        # Filter for positive track IDs
        positive_track_ids = df[df['track_ID'] > 0]['track_ID'].unique()
        
        # Print the list of unique positive track IDs
        print(f"Folder: {printrun}, Positive Track IDs: {list(positive_track_ids)}")
        if "dropRun" in printrun:
            df = feature_extraction_across_tracks(
                                                    f"/projects/steiflab/scratch/leli/trackastra/A138856A/{printrun}/tracked_again_postprocessed_2.0/man_track.txt",
                                                    f"/projects/steiflab/scratch/leli/trackastra/A138856A/{printrun}/tracked_again_postprocessed_2.0", 
                                                    f"/projects/steiflab/scratch/leli/trackastra/A138856A/{printrun}/tracked_again_postprocessed_2.0_imgs", 
                                                    output_directory=None, 
                                                    dataset="A138856A", 
                                                    display_plot=False, 
                                                    track='not all', 
                                                    track_ids=positive_track_ids,
                                                    )
            df['printrun'] = printrun
        else: 
            df = feature_extraction_across_tracks(
                                                    "/projects/steiflab/scratch/leli/trackastra/A138856A/htert_20230822_131349_843.Run/tracked_postprocessed_2.0/man_track.txt",
                                                    f"/projects/steiflab/scratch/leli/trackastra/A138856A/{printrun}/tracked_postprocessed_2.0", 
                                                    f"/projects/steiflab/scratch/leli/trackastra/A138856A/{printrun}/tracked_postprocessed_2.0_imgs", 
                                                    output_directory=None, 
                                                    dataset="A138856A", 
                                                    display_plot=False, 
                                                    track='not all', 
                                                    track_ids=positive_track_ids,
                                                    )
            df['printrun'] = printrun
        feature_metadata = pd.concat([feature_metadata, df], ignore_index=True)

    else:
        print(f"File not found: {track_to_well_path}")
feature_metadata.to_csv("/projects/steiflab/scratch/leli/A138856A/feature_metadata.csv", index = False)

## Question: What is the morphological differences between flagged and unflagged tracks in the pipeline 

In [ ]:
dataset_name = "A138856A"
printrun = "10dropRun3"
track_info_files = [
    f"/projects/steiflab/scratch/leli/trackastra/{dataset_name}/{printrun}/tracked/man_track.txt",
    f"/projects/steiflab/scratch/leli/trackastra/{dataset_name}/{printrun}/tracked_again/man_track.txt",
    f"/projects/steiflab/scratch/leli/trackastra/{dataset_name}/{printrun}/tracked_again_postprocessed/man_track.txt",
    f"/projects/steiflab/scratch/leli/trackastra/{dataset_name}/{printrun}/tracked_again_postprocessed_2.0/man_track.txt",
]
tif_directories = [
    f"/projects/steiflab/scratch/leli/trackastra/{dataset_name}/{printrun}/tracked/",
    f"/projects/steiflab/scratch/leli/trackastra/{dataset_name}/{printrun}/tracked_again/",
    f"/projects/steiflab/scratch/leli/trackastra/{dataset_name}/{printrun}/tracked_again_postprocessed/",
    f"/projects/steiflab/scratch/leli/trackastra/{dataset_name}/{printrun}/tracked_again_postprocessed_2.0/",
]
png_directories = [
    f"/projects/steiflab/scratch/leli/trackastra/{dataset_name}/{printrun}/tracked_imgs/",
    f"/projects/steiflab/scratch/leli/trackastra/{dataset_name}/{printrun}/tracked_again_imgs/",
    f"/projects/steiflab/scratch/leli/trackastra/{dataset_name}/{printrun}/tracked_again_postprocessed_imgs/",
    f"/projects/steiflab/scratch/leli/trackastra/{dataset_name}/{printrun}/tracked_again_postprocessed_2.0_imgs/",
]
output_directory = f'/projects/steiflab/scratch/leli/trackastra/feat_extract_analysis'

### This is between tracked and tracked again

In [ ]:
REMOVED_IN_TRACKED = [6, 9, 11, 15, 19, 36, 37, 42, 44, 56, 65, 69, 75, 76, 81, 83, 95, 98, 100, 103, 106, 108, 112, 116, 130, 131, 133, 137, 144, 145, 149, 152, 159, 163, 166, 181, 190, 195, 201, 204, 212, 216, 223, 225, 227, 234, 237, 240, 242, 245, 252, 255, 263, 267, 269, 270, 273, 277, 278, 287, 289, 291, 296, 297, 301, 305, 307, 309, 310, 313, 316, 319, 320, 340, 342, 350, 353, 358, 367, 369, 371, 374, 376, 378, 381, 388, 397, 400, 407, 411, 413, 421, 423, 425, 429, 430, 435, 436, 443, 445, 450, 458, 461, 462, 463]
REMOVED_IN_TRACKEDAGAIN = [65, 127, 154, 209] 
MERGED_IN_TRACKEDAGAIN = [17, 21, 31, 32, 34, 35, 36, 37, 44, 45, 53, 57, 58, 66, 68, 69, 73, 74, 94, 95, 96, 97, 107, 108, 115, 117, 118, 124, 125, 126, 131, 132, 151, 152, 153, 155, 158, 159, 160, 161, 162, 163, 167, 168, 170, 171, 175, 176, 179, 180, 182, 183, 197, 198, 200, 201, 202, 203, 204, 210, 211, 213, 214, 220, 221, 224, 225, 226, 227, 229, 230, 254, 255, 271, 272] 
REMOVED_IN_TRACKEDAGAINPP = [3, 25, 48, 54, 55, 62, 63, 71, 78, 84, 92, 101, 114, 123, 133, 140, 141, 148, 181, 191, 196, 216, 219, 228, 232, 233, 238, 239, 259, 267, 274, 279, 281, 282, 296, 319, 322, 324, 334, 341, 347, 351, 359, 361, 362, 365, 367, 371, 372, 374, 388, 392] 
MERGED_IN_TRACKEDAGAINPP = [277, 10, 9, 16, 12, 289, 290, 291, 292, 294, 298, 302, 303, 305, 308, 311, 312, 313, 314, 102, 315, 317, 318, 111, 320, 321, 323, 325, 326, 327, 328, 138, 329, 330, 143, 150, 147, 332, 336, 337, 184, 338, 339, 188, 343, 344, 194, 345, 346, 348, 349, 350, 222, 353, 354, 243, 355, 356, 357, 248, 360, 250, 363, 257, 364, 269, 368, 373, 377, 378, 379, 380, 383, 342, 256, 385, 389, 394, 395, 396] 
DIVERGED_IN_TRACKEDAGAINPP = [277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396] 

a = list(set(REMOVED_IN_TRACKEDAGAINPP) - set(DIVERGED_IN_TRACKEDAGAINPP))
#DIVERGED_IN_TRACKEDAGAINPP = list(set(DIVERGED_IN_TRACKEDAGAINPP) - set(REMOVED_IN_TRACKEDAGAINPP) - set(MERGED_IN_TRACKEDAGAINPP))
b = [
    item for item in DIVERGED_IN_TRACKEDAGAINPP 
    if item not in MERGED_IN_TRACKEDAGAINPP and item not in REMOVED_IN_TRACKEDAGAINPP
]
c = [
    item for item in MERGED_IN_TRACKEDAGAINPP 
    if item not in DIVERGED_IN_TRACKEDAGAINPP and item not in REMOVED_IN_TRACKEDAGAINPP
]

print("REMOVED_IN_TRACKED_ft")
REMOVED_IN_TRACKED_ft = feature_extraction_across_tracks(track_info_files[0], tif_directories[0], png_directories[0], output_directory = None, dataset = "A138856A", display_plot = False, track = 'not all', track_ids = REMOVED_IN_TRACKED)
print("REMOVED_IN_TRACKEDAGAIN_ft")
REMOVED_IN_TRACKEDAGAIN_ft = feature_extraction_across_tracks(track_info_files[1], tif_directories[1], png_directories[1], output_directory = None, dataset = "A138856A", display_plot = False, track = 'not all', track_ids = REMOVED_IN_TRACKEDAGAIN)
print("MERGED_IN_TRACKEDAGAIN_ft")
MERGED_IN_TRACKEDAGAIN_ft = feature_extraction_across_tracks(track_info_files[1], tif_directories[1], png_directories[1], output_directory = None, dataset = "A138856A", display_plot = False, track = 'not all', track_ids = MERGED_IN_TRACKEDAGAIN)
print("REMOVED_IN_TRACKEDAGAINPP_ft")
REMOVED_IN_TRACKEDAGAINPP_ft = feature_extraction_across_tracks(track_info_files[2], tif_directories[2], png_directories[2], output_directory = None, dataset = "A138856A", display_plot = False, track = 'not all', track_ids = a)
print("MERGED_IN_TRACKEDAGAINPP_ft")
MERGED_IN_TRACKEDAGAINPP_ft = feature_extraction_across_tracks(track_info_files[2], tif_directories[2], png_directories[2], output_directory = None, dataset = "A138856A", display_plot = False, track = 'not all', track_ids = c)
print("DIVERGED_IN_TRACKEDAGAINPP_ft")
DIVERGED_IN_TRACKEDAGAINPP_ft = feature_extraction_across_tracks(track_info_files[3], tif_directories[3], png_directories[3], output_directory = None, dataset = "A138856A", display_plot = False, track = 'not all', track_ids = b)

track_info = pd.read_csv(track_info_files[3], sep='\s+', header=None, names=['Track_ID', 'Start', 'End', 'Parent'], dtype={'Track_ID': int, 'Start': int, 'End': int, 'Parent': int})
untouched_ft = feature_extraction_across_tracks(track_info_files[3], tif_directories[3], png_directories[3], output_directory = None, dataset = "A138856A", display_plot = False, track = 'not all', track_ids = set(track_info['Track_ID'].values) - set(b))

### This is between tracked again and sub track processed 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
from cliffs_delta import cliffs_delta

# Merge the DataFrames for 'removed' and 'merged'
removed_ft = pd.concat([REMOVED_IN_TRACKED_ft, REMOVED_IN_TRACKEDAGAIN_ft, REMOVED_IN_TRACKEDAGAINPP_ft])
merged_ft = pd.concat([MERGED_IN_TRACKEDAGAIN_ft, MERGED_IN_TRACKEDAGAINPP_ft])

# Define the features to plot
features = ['area', 'circularity', 'intensity_not_norm', 'energy']

# Create a combined DataFrame with a 'source' column to differentiate between groups
def add_source_column(df, source_name):
    df['source'] = source_name
    return df

untouched_ft = add_source_column(untouched_ft, 'Untouched')
removed_ft = add_source_column(removed_ft, 'Removed')
merged_ft = add_source_column(merged_ft, 'Merged')
diverged_ft = add_source_column(DIVERGED_IN_TRACKEDAGAINPP_ft, 'Diverged')

# Combine all DataFrames
combined_df = pd.concat([untouched_ft, removed_ft, merged_ft, diverged_ft])

# Define pairs to compare for statistical tests (all vs. Untouched)
comparison_pairs = [('Untouched', 'Removed'), ('Untouched', 'Merged'), ('Untouched', 'Diverged')]

# Function to add p-value brackets
def add_brackets(ax, p_value, x1, x2, y, h, bracket_label):
    ax.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, color='black')
    ax.text((x1 + x2) * 0.5, y + h, bracket_label, ha='center', va='bottom', color='black')

# Create violin plots for each feature
for feature in features:
    plt.figure(figsize=(10, 6))
    ax = sns.violinplot(x='source', y=feature, data=combined_df)
    
    # Get the counts for each category
    source_counts = combined_df['source'].value_counts().sort_index()
    
    # Modify x-tick labels to include the counts
    xtick_labels = [f'{source} (n={source_counts[source]})' for source in source_counts.index]
    plt.xticks(ticks=range(len(xtick_labels)), labels=xtick_labels)
    
    plt.title(f'Violin Plot for {feature}')
    plt.ylabel(feature)
    plt.xlabel('Source')
    
    # Perform statistical tests and annotate the plot
    y_max = combined_df[feature].max()  # Maximum value of the feature to place annotations correctly
    h = 0.02 * y_max  # Height of the brackets
    
    # Loop through each comparison with Untouched
    for i, (group1, group2) in enumerate(comparison_pairs):
        group1_data = combined_df[combined_df['source'] == group1][feature]
        group2_data = combined_df[combined_df['source'] == group2][feature]
        
        # Mann-Whitney U test (Wilcoxon rank sum test)
        stat, p_value = mannwhitneyu(group1_data, group2_data)
        
        # Cliff's Delta Effect Size
        delta, magnitude = cliffs_delta(group1_data.tolist(), group2_data.tolist())
        
        # Set x positions for the bracket
        x1 = 0  # Untouched is always at position 0
        x2 = i + 1  # Moved by the loop index to align with Removed, Merged, and Diverged
        
        # Set y position for the bracket
        y = y_max * (1.05 + i * 0.1)
        
        # Annotate the plot with p-value and Cliff's delta
        annotation = f'p={p_value:.3e}, d={delta:.2f} ({magnitude})'
        add_brackets(ax, p_value, x1, x2, y, h, annotation)
    title = feature.capitalize()
    if feature == 'intensity_not_norm': title = "Intensity"
    if feature == 'energy': title = "Textual Homogeneity"
    plt.savefig(f"/projects/steiflab/scratch/leli/A138856A/plots/{title}_tracking_processed_objects_violinplot.png")
    plt.show()


### This is between sub track processed and all track processed

In [ ]:
DIVERGED_IN_TRACKEDAGAINPP_ft.columns